In [ ]:
!pip install -U opencv-python numpy pillow pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60.9 MB 1.3 MB/s 
     |████████████████████████████████| 3.1 MB 38.7 MB/s 
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import shutil
import csv
import cv2
from PIL import Image
import numpy as np
from pathlib import Path

In [ ]:
# Monta os arquivos do google drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
base_dir = Path('/content/drive/My Drive/TCC/')
base_dir.cwd()

Mounted at /content/drive/


PosixPath('/content')

In [ ]:
def average_distance(img1, img2):
    return np.average(img1 - img2)

def l2_distance(img1, img2):
    width, height = img1.shape[:2]
    return np.linalg.norm(img1 - img2) / (width * height)

def structure_content(img1, img2):
    return np.sum(np.square(img1)) / np.sum(np.square(img2))

def image_fidelity(img1, img2):
    return 1 - np.sum(np.square(img1 - img2)) / np.sum(np.square(img2))

def n_cross_correlation(img1, img2):
    return np.sum(img1 * img2) / np.sum(np.square(img1))

def nmse(img1, img2):
    return np.sum(np.square(img1 - img2)) / np.sum(np.square(img1))

def psnr(img1, img2):
    return cv2.PSNR(img1, img2)

In [ ]:
temp_dir = Path('/temp_jpg')

def create_jpg_copy(png_file):
  filename = png_file.stem
  img = Image.open(png_file)
  output_path = temp_dir / f'{filename}.jpg'
  img.save(output_path)
  return output_path


In [ ]:
def write_features_and_labels(folder):
  if temp_dir.exists():
    shutil.rmtree(temp_dir)
  temp_dir.mkdir()

  csv_path = base_dir / f'{folder.name}_features_label.csv'
  if csv_path.exists():
    csv_path.unlink()

  with csv_path.open('w', newline = '') as csv_file:
    writer = csv.writer(csv_file)
    for child in folder.iterdir():
      if child.suffix != '.png':
        continue
      print(f'Processing file {child.as_posix()}')
      jpg_copy_path = create_jpg_copy(child)
      img1 = cv2.imread(child.as_posix())
      img2 = cv2.imread(jpg_copy_path.as_posix())

      ad = average_distance(img1, img2)
      l2dist = l2_distance(img1, img2)
      struct_content = structure_content(img1, img2)
      fidelity = image_fidelity(img1, img2)
      n_cross_rel = n_cross_correlation(img1, img2)
      nmse_val = nmse(img1, img2)
      psnr_val = psnr(img1, img2)

      filename = child.stem

      if filename.startswith('encoded_'):
        label = 1
      else:
        label = 0
      
      writer.writerow([filename, ad, l2dist, struct_content, fidelity, n_cross_rel, nmse_val, psnr_val, label])

In [ ]:
#TARGET_CAPACITIES = [10, 25, 50, 75, 100]
TARGET_CAPACITIES = [100]
for capacity in TARGET_CAPACITIES:
  dataset_folder = base_dir / f"lsb_{capacity}"
  print(f'writing labels and features for {dataset_folder.name}')
  write_features_and_labels(dataset_folder)

Streaming output truncated to the last 5000 lines.
Processing file /content/drive/My Drive/TCC/lsb_100/12627.png
Processing file /content/drive/My Drive/TCC/lsb_100/10390.png
Processing file /content/drive/My Drive/TCC/lsb_100/11223.png
Processing file /content/drive/My Drive/TCC/lsb_100/14757.png
Processing file /content/drive/My Drive/TCC/lsb_100/14712.png
Processing file /content/drive/My Drive/TCC/lsb_100/13914.png
Processing file /content/drive/My Drive/TCC/lsb_100/15141.png
Processing file /content/drive/My Drive/TCC/lsb_100/13293.png
Processing file /content/drive/My Drive/TCC/lsb_100/10032.png
Processing file /content/drive/My Drive/TCC/lsb_100/10237.png
Processing file /content/drive/My Drive/TCC/lsb_100/12463.png
Processing file /content/drive/My Drive/TCC/lsb_100/14580.png
Processing file /content/drive/My Drive/TCC/lsb_100/14848.png
Processing file /content/drive/My Drive/TCC/lsb_100/11539.png
Processing file /content/drive/My Drive/TCC/lsb_100/11605.png
Processing file /co